In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# data = pd.read_csv('/kaggle/input/icc-2000-to-9000-new/icc_2000_to_9000.csv')
data = pd.read_csv('/kaggle/input/icc-2000-to-3000/icc_2000_to_3000.csv')
data.shape

(982, 11)

In [3]:
extract = data.iloc[1:,:]

failed = []
questions = ['name', 'complaining_party', 'defending_party', 'judges','court','end_date']
context_df = []
answers_df = []
questions_df = []
for index,row in extract.iterrows():
    current_row = []
    context = row.text.replace('_:::_',' ')
#     print(index,context)
    for i,question in enumerate(questions):
        
        if question == 'end_date':
            answer = str(row[question]).strip()
            try:
                year,month,day = answer.split("-")
            except:
                print(answer)
                continue
            if month[0] == '0': month = month[1]
            if day[0] == '0': day = day[1]
            year_start_idx = context.find(year)
            year_end_idx = context.find(year) + len(year)
            if year == context[int(year_start_idx):int(year_end_idx)]:
                context_df.append(str(context))
                answers_df.append({'text': year,'s':year_start_idx,'e': year_end_idx})
                questions_df.append('year')
            else: failed.append('year')
#             print("year",year_start_idx, year == context[int(year_start_idx):int(year_end_idx)])
            month_start_idx = context.find(month)
            month_end_idx = context.find(month) + len(month)
            if month == context[int(month_start_idx):int(month_end_idx)]:
                context_df.append(str(context))
                answers_df.append({'text': month,'s':month_start_idx,'e': month_end_idx})
                questions_df.append('month')
            else: failed.append('month')
#             print("month",month_start_idx, month == context[int(month_start_idx):int(month_end_idx)])
            day_start_idx = context.find(day)
            day_end_idx = context.find(day) + len(day)
            if day == context[int(day_start_idx):int(day_end_idx)]:
                context_df.append(str(context))
                answers_df.append({'text': day,'s':day_start_idx,'e': day_end_idx})
                questions_df.append('day')
            else: failed.append('day')
#             print("day",day_start_idx, day == context[int(day_start_idx):int(day_end_idx)])
        elif question == 'judges':
            lowest_index = 9999
            highest_index = 0
            judges = str(row[question]).replace(",","").replace("|"," ")
            for judge in judges.split(" "):
                if not judge: continue
                judge = judge.strip()
                dex = context.find(judge)
                ldex = context.rfind(judge)
                if dex != -1:
                    lowest_index = dex if dex < lowest_index else lowest_index
                    highest_index = ldex + len(judge) if ldex > highest_index else highest_index
            if lowest_index == 9999 and highest_index == 0: continue
            start_idx = lowest_index
            end_idx = highest_index
#             print(question,start_idx, end_idx, context[int(start_idx):int(end_idx)])
#             if day == context[int(day_start_idx):int(day_end_idx)]:
            context_df.append(str(context))
            answers_df.append({'text': context[int(start_idx):int(end_idx)],'s':start_idx,'e': end_idx})
            questions_df.append(str(question))
#             else: failed.append(question)
            
        else:
            answer = str(row[question]).strip()
            start_idx = context.find(answer)
            end_idx = start_idx + len(answer)
            if answer == context[int(start_idx):int(end_idx)]:
                context_df.append(str(context))
                answers_df.append({'text': answer,'s':start_idx,'e': end_idx})
                questions_df.append(str(question))
            else: failed.append(question)
#             print(question,start_idx, answer == context[int(start_idx):int(end_idx)])
        

    

print(len(context_df), len(answers_df), len(questions_df))
questions.remove('end_date')
questions.append('year')
questions.append('month')
questions.append('day')
for q in questions:
    sc = questions_df.count(q)
    fc = failed.count(q)
    total = sc+ fc
    print(f'Success for {q} is {sc/total *100} good {sc} bad {fc} total {total}')


7458 7458 7458
Success for name is 82.97655453618756 good 814 bad 167 total 981
Success for complaining_party is 95.61671763506627 good 938 bad 43 total 981
Success for defending_party is 87.86952089704383 good 862 bad 119 total 981
Success for judges is 100.0 good 977 bad 0 total 977
Success for court is 97.65545361875637 good 958 bad 23 total 981
Success for year is 98.77675840978594 good 969 bad 12 total 981
Success for month is 99.18450560652396 good 973 bad 8 total 981
Success for day is 98.57288481141691 good 967 bad 14 total 981


In [4]:
print(context_df[0],answers_df[0],questions_df[0])

AIR 1950 MADHYA BHARAT 39 MADHYA BHARAT HIGH COURT  REGE   J.  Second Appeal No.227 of 1948  D/- 9 - 4 - 1949  Jugrajsa Chunilalsa Appellants v. Umrao Singh Sikdarsingh and others Respondents  Transfer of Property Act (4 of 1882)  S.108(h)  S.108(o) - Trees planted by tenant - Right to.  Samvatsar - for Appellant. VIPIN SANGHI - for Respondents.  Appeal Allowed .  {'text': 'Jugrajsa Chunilalsa Appellants v. Umrao Singh Sikdarsingh and others', 's': 111, 'e': 179} name


In [5]:
from sklearn.model_selection import train_test_split
train_contexts, val_contexts, train_questions, val_questions, train_answers, val_answers = train_test_split(context_df, questions_df, answers_df, random_state=3, test_size=0.2)
print(len(train_contexts), len(val_contexts), len(train_questions), len(val_questions), len(train_answers), len(val_answers))


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


5966 1492 5966 1492 5966 1492


In [6]:
print(val_contexts[0],val_answers[0],val_questions[0])

AIR 1951 CALCUTTA 418 CALCUTTA HIGH COURT  DAS GUPTA   J. and P. N. MOOKERJEE   J.  Criminal Revn. No. 591 of 1950  D/- 5 - 1 - 1951  Bepin Behari Maity - Complainant - Petnr v. Paban Sardar and other Accused Opposite Party  Criminal P.C. (5 of 1898)  S.256  S.258  S.439 - Duty of Magistrate u/S.256 - Non-compliance with - Order of acquittal - Legality.  Anno. Cr. P. C.  S. 256  N. 4  5; S. 258  N. 4; S. 439  N. 12.  Nalin Chandra Banerjee for J.M. Banerjee - for Petnr.; Nalini Kumar Mukherjee - for Opposite Party.  Retrial Ordered .  {'text': 'DAS GUPTA   J. and P. N. MOOKERJEE   J.  Criminal Revn. No. 591 of 1950  D/- 5 - 1 - 1951  Bepin Behari Maity - Complainant - Petnr v. Paban Sardar and other Accused Opposite Party  Criminal P.C. (5 of 1898)  S.256  S.258  S.439 - Duty of Magistrate u/S.256 - Non-compliance with - Order of acquittal - Legality.  Anno. Cr. P. C.  S. 256  N. 4  5; S. 258  N. 4; S. 439  N. 12.  Nalin Chandra Banerjee for J.', 's': 43, 'e': 450} judges


In [7]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

In [8]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['s']))
        end_positions.append(encodings.char_to_token(i, answers[i]['e'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [9]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: train_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: train_encodings[key] for key in ['start_positions', 'end_positions']}
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    {key: val_encodings[key] for key in ['input_ids', 'attention_mask']},
    {key: val_encodings[key] for key in ['start_positions', 'end_positions']}
)) 

In [10]:
from transformers import TFDistilBertForQuestionAnswering
model = TFDistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [11]:
# Keras will expect a tuple when dealing with labels
train_dataset = train_dataset.map(lambda x, y: (x, (y['start_positions'], y['end_positions'])))
# Keras will assign a separate loss for each output and add them together. So we'll just use the standard CE loss
# instead of using the built-in model.compute_loss, which expects a dict of outputs and averages the two terms.
# Note that this means the loss will be 2x of when using TFTrainer since we're adding instead of averaging them.
# detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# instantiate a distribution strategy
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

# with tpu_strategy.scope():
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.distilbert.return_dict = False # if using 🤗 Transformers >3.02, make sure outputs are tuples
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) # can also use any keras loss fn
    
records = model.fit(train_dataset.shuffle(1000).batch(16), epochs=2, batch_size=16)

Epoch 1/2
373/373 [==============================] - 161s 349ms/step - loss: 6.0744 - output_1_loss: 2.8492 - output_2_loss: 3.2253 - output_1_accuracy: 0.1780 - output_2_accuracy: 0.1353
Epoch 2/2
373/373 [==============================] - 124s 333ms/step - loss: 2.3825 - output_1_loss: 0.9964 - output_2_loss: 1.3861 - output_1_accuracy: 0.6559 - output_2_accuracy: 0.5575


In [12]:
model.save('basic2e')

# Test

In [13]:
context_test = ["AIR 1951 CALCUTTA 418 CALCUTTA HIGH COURT  DAS GUPTA   J. and P. N. MOOKERJEE   J.  Criminal Revn. No. 591 of 1950  D/- 5 - 1 - 1951  Bepin Behari Maity - Complainant - Petnr v. Paban Sardar and other Accused Opposite Party  Criminal P.C. (5 of 1898)  S.256  S.258  S.439 - Duty of Magistrate u/S.256 - Non-compliance with - Order of acquittal - Legality.  Anno. Cr. P. C.  S. 256  N. 4  5; S. 258  N. 4; S. 439  N. 12.  Nalin Chandra Banerjee for J.M. Banerjee - for Petnr.; Nalini Kumar Mukherjee - for Opposite Party.  Retrial Ordered ."]
question_test = ["name"]

encoded_test = tokenizer(context_test,question_test, padding=True, truncation=True)
print(encoded_test)

{'input_ids': [[101, 19016, 2069, 3280, 8784, 12674, 16830, 9159, 3746, 1604, 8784, 12674, 16830, 9159, 145, 23413, 3048, 18732, 19556, 1942, 141, 10719, 144, 18124, 9159, 147, 119, 1105, 153, 119, 151, 119, 150, 2346, 22027, 9637, 4538, 27073, 147, 119, 10382, 6750, 1179, 119, 1302, 119, 4589, 1475, 1104, 3067, 141, 120, 118, 126, 118, 122, 118, 3280, 4108, 6709, 4108, 16234, 17551, 2340, 118, 3291, 8223, 15858, 2861, 118, 25993, 1179, 1197, 191, 119, 19585, 7167, 17784, 18484, 1197, 1105, 1168, 138, 19515, 11031, 9126, 5674, 13068, 1786, 10382, 153, 119, 140, 119, 113, 126, 1104, 5381, 114, 156, 119, 18440, 156, 119, 27434, 156, 119, 3887, 1580, 118, 21134, 1104, 7085, 25019, 18775, 190, 120, 156, 119, 18440, 118, 7922, 118, 14037, 1114, 118, 2864, 1104, 170, 1665, 18276, 5100, 1233, 118, 10800, 1785, 119, 5083, 1186, 119, 140, 1197, 119, 153, 119, 140, 119, 156, 119, 18440, 151, 119, 125, 126, 132, 156, 119, 27434, 151, 119, 125, 132, 156, 119, 3887, 1580, 151, 119, 1367, 119, 11896

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: encoded_test[key] for key in ['input_ids', 'attention_mask']}))
print(train_dataset)

<_TensorSliceDataset element_spec={'input_ids': TensorSpec(shape=(211,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(211,), dtype=tf.int32, name=None)}>


In [15]:
outputs = model.predict(train_dataset)


1/1 [==============================] - 1s 1s/step


In [16]:
start_logits, end_logits = outputs['start_logits'], outputs['end_logits']
start_index = tf.argmax(start_logits, axis=1).numpy()[0]
end_index = tf.argmax(end_logits, axis=1).numpy()[0] + 1 
print(start_index, end_index)
tokens = tokenizer.convert_ids_to_tokens(encoded_test['input_ids'][0])
answer_tokens = tokens[start_index:end_index]
answer = tokenizer.convert_tokens_to_string(answer_tokens)
print(answer)

58 89
Bepin Behari Maity - Complainant - Petnr v. Paban Sardar and other Accused Opposite Party
